In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Preprocessing

In [3]:
df_movies=pd.read_csv('./ml-latest-small/ml-latest-small/movies.csv')

In [4]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_ratings=pd.read_csv('./ml-latest-small/ml-latest-small/ratings.csv')


In [6]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
transactional_dataset = df_ratings[df_ratings['rating']>2]

In [32]:
data_set={}
for userid in transactional_dataset['userId'].unique():
    if userid not in data_set:
        data_set[userid]=set()
    temp_df=transactional_dataset[transactional_dataset['userId']==userid]
    for i in range(len(temp_df)):
        data_set[userid].add(temp_df.iloc[i]['movieId'])
    if len(data_set[userid])<=10:
        del data_set[userid]

In [35]:
import random
training_set={}
testing_set={}
for key in data_set.keys():
    lst=list(data_set[key])
    n=len(lst)
    train_len=int(n*0.8)
    test_len=n-train_len
    random.shuffle(lst)
    if key not in training_set:
        training_set[key]=set()
        train_lst=set(lst[:train_len])
        training_set[key]=train_lst
    if key not in testing_set:
        testing_set[key]=set()
        test_lst=set(lst[train_len:])
        testing_set[key]=test_lst


### Association Rule Mining

In [37]:
dummy={1:{1,2,5},2:{2,4},3:{2,3},4:{1,2,4},5:{1,3},6:{2,3},7:{1,3},8:{1,2,3,5},9:{1,2,3}}

In [41]:
dummy

{1: {1, 2, 5},
 2: {2, 4},
 3: {2, 3},
 4: {1, 2, 4},
 5: {1, 3},
 6: {2, 3},
 7: {1, 3},
 8: {1, 2, 3, 5},
 9: {1, 2, 3}}

In [143]:
min_sup=2 
min_conf=0.1

In [42]:
L={}
for key in dummy.keys():
    se=dummy[key]
    for val in se:
        if val not in L:
            L[val]=1
        else:
            L[val]+=1
print(L)


{1: 6, 2: 7, 5: 2, 4: 2, 3: 6}


In [43]:
#now we need to sort the dictionary in descending order
L = dict(sorted(L.items(),key=lambda item: item[1],reverse=True))
print(L)

{2: 7, 1: 6, 3: 6, 5: 2, 4: 2}


In [114]:
class TreeNode:
    def __init__(self,id=-1,cnt=0):
        self.id=id
        self.cnt=cnt
        self.pointers={} #stores key id and value is reference to a node 

    def check(self,node):
        if node not in self.pointers:
            return False
        return True
    def get(self,node):
        return self.pointers[node]
    

In [145]:
class Tree:
    def __init__(self):
        self.root=TreeNode()
        self.par={} # node to node
        self.dic={}
    def insert(self,lst,inc=1):
        temp=self.root
        prev=None
        for x in lst:
            if temp.check(x)==False:
                temp.pointers[x]=TreeNode(x,0)
            self.par[temp]=prev
            prev=temp
            temp=temp.get(x)
            if temp.id not in self.dic:
                self.dic[temp.id]=set()
            self.dic[temp.id].add(temp)
            temp.cnt+=inc
    def show(self,root):
        for key in root.pointers.keys():
            temp=root.pointers[key]
            st="id "+str(temp.id)+" "+"cnt "+str(temp.cnt)
            print(st)
            self.show(temp)

    def helper_path(self,node,path,item,ans):
        if node.id==item:
            ans[tuple(path)]=node.cnt
            return
        for key,value in node.pointers.items():
            path.append(key)
            self.helper_path(value,path,item,ans)
            path.pop()

    def get_paths(self,item):
        #gets paths to that node
        ans={} # key is set and value is cnt
        temp=self.root
        path=[]
        for key in temp.pointers.keys():
            path.append(key)
            self.helper_path(temp.pointers[key],path,item,ans)
            path.pop()
        return ans
            
    def print_par(self):
        for key,val in self.par.items():
            if key is not None and val is not None:
                print(key.id,val.id)
    

In [146]:
#create a root node
tree=Tree()
#now iterate through the transactions (ie dummy) and insert the items in the order of L
order_of_insertion=list(L.keys())
#print(order_of_insertion)
for key,values in dummy.items():
    # insert the values in the values in the order of L
    lst=list(values)
    ordered_lst=sorted(lst,key=lambda x: order_of_insertion.index(x))
    #print(ordered_lst)
    tree.insert(ordered_lst)

tree.show(tree.root)

id 2 cnt 7
id 1 cnt 4
id 5 cnt 1
id 4 cnt 1
id 3 cnt 2
id 5 cnt 1
id 4 cnt 1
id 3 cnt 2
id 1 cnt 2
id 3 cnt 2


In [152]:
# now creating conditional pattern base
# so for every path ending with item push the path into a list
def build_L(conditional_pattern_base):
    L_conditional={}
    def fill_L(itemset,sup_count):
        for item in itemset:
            if item not in L_conditional:
                L_conditional[item]=sup_count
            else:
                L_conditional[item]+=sup_count

    for itemset,sup_count in conditional_pattern_base.items():
        lst=list(itemset)
        lst.pop()
        fill_L(lst,sup_count)
    keys_to_delete=[]
    for key, cnt in L_conditional.items():
        if cnt<min_sup:
            keys_to_delete.append(key)
    for key in keys_to_delete:
        if key in L_conditional:
            del L_conditional[key]
    return L_conditional

temp_lst=list(L.keys())
for item in reversed(temp_lst):
    #for the item 
    conditional_pattern_base=tree.get_paths(item)
    L_conditional=build_L(conditional_pattern_base)
    #now sort the L in descending order
    L_conditional = dict(sorted(L_conditional.items(),key=lambda item: item[1],reverse=True))
    order_of_insertion=list(L_conditional.keys())
    tree_conditional=Tree()
    for itemset,sup_count in conditional_pattern_base.items():
        lst=list(itemset)
        lst.pop()
        for x in lst:
            if x not in order_of_insertion:
                lst.remove(x)
        ordered_lst=sorted(lst,key=lambda x: order_of_insertion.index(x))
        tree_conditional.insert(lst,sup_count)
    #print(L_conditional)
    print("for item ",item)
    tree_conditional.show(tree_conditional.root)

    #after building the tree we need to extract frequent patterns
    


for item  4
id 2 cnt 2
for item  5
id 2 cnt 2
id 1 cnt 2
for item  3
id 2 cnt 4
id 1 cnt 2
id 1 cnt 2
for item  1
id 2 cnt 4
for item  2
